In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# **Dataset**
1. Jester Dataset 1 contains 24983 rows and 101 columns.
2. Each row corresponds to a user.
3. First column is the number of jokes rated by that user.
4. The numbers refer to the ID's of the jokes in the Excel files.
5. Others are ratings for each joke. Ratings are in the range of -10 to 10. Value of 99 indicates Null, which is no rating.


In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/zeitgeist-hash/CE888/main/Lab_4/jester-data-1.csv", header=None)
print(df)
print(df.info())

       0      1      2      3      4    ...    96     97     98     99     100
0       74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24978  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24979   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24980   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24982   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 101 columns]
<class 'pandas.core.fram

In [ ]:
df = df.loc[:, 1:]
arr = df.values
print(arr.shape)

(24983, 100)


In [ ]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1810455 (1810455,)


In [ ]:
df_joke = df
df_joke[df_joke==99] = np.nan
df_joke

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.42,NaN,NaN,NaN,-7.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,7.91,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,NaN,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,NaN,8.25,NaN,NaN,7.48,7.28,7.28,8.93,NaN,6.17,7.28,NaN,NaN,8.98,7.33,NaN,6.17,9.08,7.33,...,6.46,7.28,NaN,NaN,7.04,7.28,NaN,7.28,8.25,NaN,NaN,NaN,NaN,NaN,NaN,8.93,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.03,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,NaN,NaN,NaN,NaN,2.91,3.93,6.75,6.60,NaN,6.65,-6.12,NaN,7.57,6.21,6.65,NaN,-8.30,7.18,2.82,...,0.00,-3.69,NaN,NaN,7.82,0.24,NaN,7.28,-2.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.63,NaN,NaN,-2.33,NaN,NaN,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,2.48,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,NaN,NaN,NaN,NaN,NaN,NaN,4.13,NaN,NaN,NaN,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,-0.53,5.63,-7.14,-4.08,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46,5.63,-0.92,-2.91,-4.17,2.82,3.40,8.64,6.84,6.80,-0.87,7.38,-3.50,8.88,7.43,5.39,2.23,-0.68,3.40,-0.58,4.42,...,8.59,3.45,0.87,9.27,-4.66,5.73,-0.49,8.35,1.94,5.00,-9.66,8.98,8.98,-9.81,9.13,9.08,9.08,3.98,0.73,9.03,8.98,9.22,8.93,9.13,9.27,-1.99,-9.95,-9.90,9.13,8.83,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,8.88,-8.69,-7.48,-8.83,-1.75,6.60,3.54,1.50,7.67,-0.44,9.22,8.74,9.03,9.08,8.93,3.74,3.20,-9.17,-8.98,8.79,-7.67,-3.06,9.13,8.40,-0.63,-7.18,0.58,8.88,9.27,8.50,...,2.77,8.11,-7.96,8.93,-0.87,-5.87,8.88,-1.12,-8.74,8.74,NaN,NaN,NaN,NaN,NaN,4.90,NaN,NaN,NaN,NaN,-0.29,0.92,-0.78,0.15,-0.10,0.00,-0.19,-0.87,-1.36,-0.58,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,NaN,NaN,NaN,NaN,-7.77,NaN,6.70,-6.75,NaN,NaN,NaN,NaN,-6.46,-1.65,-6.80,-6.41,-6.99,7.23,6.75,-6.99,6.55,NaN,NaN,NaN,NaN,0.49,-0.53,-6.94,-0.49,NaN,6.46,-0.53,NaN,NaN,-7.86,-0.34,NaN,-6.94,NaN,NaN,...,0.49,-0.24,NaN,NaN,-3.11,-6.65,NaN,-0.58,6.31,NaN,NaN,-7.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24981,NaN,NaN,NaN,NaN,-9.71,NaN,4.56,-8.30,NaN,NaN,NaN,NaN,-9.47,NaN,3.45,-0.92,-4.51,-4.13,-5.73,-9.51,2.82,NaN,NaN,NaN,NaN,-0.49,2.91,2.62,8.30,NaN,3.06,5.44,NaN,NaN,-0.68,2.04,NaN,NaN,1.

In [ ]:
# Question 3
df_joke.iloc[:,:].mean().sort_values()

58   -3.833880
16   -3.104818
44   -2.111891
57   -1.990664
13   -1.762882
        ...   
32    3.155988
27    3.189821
36    3.312500
89    3.574720
50    3.665085
Length: 100, dtype: float64

In [ ]:
# Question 4
df_joke.iloc[12761, 58]

3.83

In [ ]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [ ]:
new_arr, idx = replace(arr, 0.1)

In [ ]:
arr[idx[0][0], idx[1][0]]

8.83

In [ ]:
new_arr[idx[0][0], idx[1][0]]

99.0

# **Latent Factor**

In [ ]:
#Set latent factors as 50
n_latent_factors = 50

user_ratings = new_arr

# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [ ]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001, weight_decay=0.1):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * (latent_item_features[item_id])
    latent_item_features[item_id] -= alpha * err * (user_pref_values)
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating!=99):
                # if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        # if (iteration % 100) == 0:
        #     print('MSE ', iteration, ': ', mse)
        print('MSE ', iteration, ': ', mse)

In [ ]:
num_iter = 50
sgd(num_iter)

MSE  0 :  28.638563547765674
MSE  1 :  25.036577403845012
MSE  2 :  24.950843168405733
MSE  3 :  24.88232813862681
MSE  4 :  24.822793425183622
MSE  5 :  24.765265482877627
MSE  6 :  24.702795996820964
MSE  7 :  24.627318347994162
MSE  8 :  24.528461721204284
MSE  9 :  24.392325779340705
MSE  10 :  24.20048317843752
MSE  11 :  23.929975788875364
MSE  12 :  23.55580667700697
MSE  13 :  23.05791758616145
MSE  14 :  22.433372014292814
MSE  15 :  21.709528495516462
MSE  16 :  20.947186128613307
MSE  17 :  20.223739555739016
MSE  18 :  19.602706013196748
MSE  19 :  19.112016727293334
MSE  20 :  18.745335107225046
MSE  21 :  18.478260531883635
MSE  22 :  18.283609465275646
MSE  23 :  18.13889671858224
MSE  24 :  18.0277760161413
MSE  25 :  17.938983564781378
MSE  26 :  17.86486609593892
MSE  27 :  17.80019402804844
MSE  28 :  17.74134243893592
MSE  29 :  17.685755342889532
MSE  30 :  17.63160041546766
MSE  31 :  17.577546704181152
MSE  32 :  17.522621287937707
MSE  33 :  17.466116798607366
M

In [ ]:
def validation_check():
  error = []
  prediction = latent_user_preferences.dot(latent_item_features.T)
  for i in range(len(idx[0])):
    predicted_rating = prediction[idx[0][i]][idx[1][i]]
    actual_rating = arr[idx[0][i]][idx[1][i]]
    err = predicted_rating - actual_rating
    error.append(err)
  mse = (np.array(error) ** 2).mean()
  print('Validation MSE: ', mse)

In [ ]:
validation_check()

Validation MSE:  17.815538549236273


In [ ]:
# Creating new validation arrays from original data randomly again
for i in range(10):
  new_arr, idx = replace(arr, 0.1)
  validation_check()

Validation MSE:  16.284825032090804
Validation MSE:  16.43737996935927
Validation MSE:  16.31993557088579
Validation MSE:  16.431960569166325
Validation MSE:  16.46936076758466
Validation MSE:  16.341887194289257
Validation MSE:  16.405127686515147
Validation MSE:  16.41729826580208
Validation MSE:  16.32430044037324
Validation MSE:  16.50220766751229


# Results






In [ ]:
prediction = latent_user_preferences.dot(latent_item_features.T)

In [ ]:
# Prediction values on entire Test Dataset

values = [zip(df.values[i], prediction[i]) for i in range(prediction.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction
comparison_data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, -3.7179174396851313)","(8.79, -1.6132444596295863)","(-9.66, -6.587026600558394)","(-8.16, -8.864964180605547)","(-7.52, -2.7323146101562927)","(-8.5, -2.708623468785011)","(-9.85, -3.5951564786541055)","(4.17, -4.003113626824221)","(-8.98, -7.9693652067344845)","(-4.76, -4.919980034299686)","(-8.5, -3.5761238129057493)","(-6.75, -2.505636192703524)","(-7.18, -3.9944529755579827)","(8.45, -1.430567115012328)","(-7.18, -3.9616852433515035)","(-7.52, -6.891802814448163)","(-7.43, -3.5222614611658978)","(-9.81, -3.625597106918512)","(-9.85, -3.4335150883778143)","(-9.85, -5.577780337592808)","(-9.37, -0.47756755482668883)","(1.5, -3.9585494514359847)","(-4.37, -5.757736163807887)","(-9.81, -9.506376528231172)","(-8.5, -3.1847762254778873)","(1.12, -1.4921313302649524)","(7.82, 0.9427077352438069)","(2.86, -3.0282886100508106)","(9.13, 2.0361479362418704)","(-7.43, -7.977285791390199)","(2.14, -0.29520733741846666)","(-4.08, 1.0026230478286706)","(-9.08, -8.402045804084873)","(7.82, -2.548109593361306)","(5.05, 0.7680408561992815)","(4.95, 1.2587310625607029)","(-9.17, -7.277387738399995)","(-8.4, -4.201445708135123)","(-8.4, -3.3101359207549423)","(-8.4, -3.644299161337537)",...,"(8.59, -1.0423302340920166)","(3.59, -0.38571266538986226)","(-6.84, -5.240647060688812)","(-9.03, -7.476561846529644)","(2.82, 0.17531410812477904)","(-1.36, -0.22208032985391793)","(-9.08, -7.4287706183454585)","(8.3, -0.38927128908098685)","(5.68, 0.6217134850804468)","(-4.81, -4.585389904387626)","(99.0, -5.413351852796263)","(99.0, -2.102184901267556)","(99.0, -6.200179563616615)","(99.0, -8.297481201322736)","(99.0, -8.41360065112378)","(99.0, -2.2883821210874173)","(99.0, -6.517700076249997)","(-9.42, -5.74819351077247)","(99.0, -7.209475504269819)","(99.0, -5.075133796094891)","(99.0, -2.5799732122615624)","(-7.72, -5.357751559563038)","(99.0, -3.1204707988460543)","(99.0, -5.784226368622381)","(99.0, -6.0883871532247955)","(99.0, -6.171643276871328)","(99.0, -3.2991986810649223)","(99.0, -3.093584699665226)","(99.0, 0.1523183449095052)","(99.0, -6.354573237915222)","(2.82, -2.6709180172589866)","(99.0, -4.512322109622376)","(99.0, -2.597526526706805)","(99.0, -5.581896127694096)","(99.0, -5.4518429433056745)","(99.0, -4.196133425529526)","(-5.63, -3.5358045400962466)","(99.0, -3.868456786358891)","(99.0, -6.636807414403714)","(99.0, -4.79978010556495)"
1,"(4.08, 3.778234904017037)","(-0.29, 0.8569751976491499)","(6.36, 4.2417603986677594)","(4.37, 4.206918167579914)","(-2.38, 0.15439316659480531)","(-9.66, 2.0613067926144932)","(-0.73, 1.219695202804141)","(-5.34, 1.3272211377288876)","(8.88, 5.628604394518986)","(9.22, 5.184899420810397)","(6.75, 2.7291886466158592)","(8.64, 1.8343534414535725)","(4.42, -0.04688635076108584)","(7.43, 1.1722165273652387)","(4.56, -0.13408784138851446)","(-0.97, 1.6152784984881383)","(4.66, -0.1987955143020759)","(-0.68, 0.973138558203871)","(3.3, 1.8778456855436636)","(-1.21, 1.1860046306631322)","(0.87, 1.0410708005956222)","(8.64, 3.2486154643208485)","(8.35, 4.5955160502113594)","(9.17, 6.136983372521145)","(0.05, 2.087628108610292)","(7.57, 0.7422271202531185)","(4.71, 1.3903638602077044)","(0.87, 1.8651699123988092)","(-0.39, 0.3130115744052067)","(6.99, 3.997240199211408)","(6.5, 0.8831413622525389)","(-0.92, 1.5573920068899925)","(7.14, 4.045651937973875)","(9.03, 2.4149916138259466)","(-1.8, 1.4728550336431858)","(0.73, 1.310749716936902)","(7.09, 2.558462547892566)","(3.4, 2.965931236303718)","(-0.87, 1.8569361367958228)","(7.91, 2.0301302746312286)",...,"(-6.7, 0.3411849434479511)","(-3.35, 1.5397992157950304)","(-9.03, 2.3155030545752338)","(4.47, 4.951082678328272)","(4.08, 0.7164417073403598)","(-3.83, 1.2004208815437338)","(8.74, 6.502119192635598)","(1.1

In [ ]:
jokes_ratings = pd.DataFrame(prediction)
mean_idx = jokes_ratings.mean()
min_idx = mean_idx.idxmin()
max_idx = mean_idx.idxmax()
print('Worst Joke ID predicted: ', min_idx, '; Average Rating: ', mean_idx[min_idx])
print('Best Joke ID predicted: ', max_idx, '; Average Rating: ', mean_idx[max_idx])

Worst Joke ID predicted:  57 ; Average Rating:  -4.111693292049609
Best Joke ID predicted:  49 ; Average Rating:  3.521651851012798


# Movie Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_items = pd.read_excel('/content/drive/MyDrive/CE888 Data Science for Decision Making/movies_latent_factors.xlsx', sheet_name='Items') 
df_items

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9806,The Incredibles (2004),0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,10020,Beauty and the Beast (1991),1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,36657,X-Men (2000),0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,36658,X2: X-Men United (2003),1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130


In [ ]:
df_users = pd.read_excel('/content/drive/MyDrive/CE888 Data Science for Decision Making/movies_latent_factors.xlsx', sheet_name='Users') 
df_users

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,4768,-0.204024,0.161079,-0.090447,0.138495,-0.162934,0.163894,0.051502,-0.088582,0.126829,0.065967,0.085008,0.355404,0.007108,-0.118663,-0.039125
1,156,-0.189652,-0.178979,-0.091490,-0.000823,-0.032646,0.177209,-0.098123,-0.068283,-0.011575,0.120866,-0.009931,-0.048606,0.045916,0.113671,0.179873
2,5323,-0.115308,-0.090886,-0.053129,0.018472,-0.068081,-0.004828,0.113005,0.102107,0.034758,0.000693,-0.073712,-0.019460,0.108372,0.054471,-0.109552
3,174,-0.227462,-0.272532,-0.017231,0.054324,0.214755,-0.072639,-0.033122,-0.086508,-0.131479,0.180403,0.095890,-0.082396,0.036767,-0.165438,0.050692
4,4529,-0.014616,-0.102218,-0.107935,0.155784,-0.123362,-0.118228,-0.013549,-0.050622,0.058698,-0.159600,-0.142382,-0.132836,-0.039897,0.129063,0.102669
5,783,-0.020301,-0.031919,-0.036955,0.033690,0.000174,-0.003178,0.054474,0.045424,-0.053308,0.003437,-0.041750,0.059725,-0.021647,0.039873,-0.061857
6,3878,-0.091462,0.215879,-0.180453,0.085408,-0.321094,0.227947,0.053767,0.038017,0.097141,-0.139872,0.029253,0.214467,0.189250,0.065990,0.102776
7,768,0.000819,-0.009229,-0.019228,0.002703,0.012869,0.006655,0.007687,0.018747,0.014705,-0.009256,-0.010116,-0.010051,-0.002108,0.043223,-0.006670
8,4469,-0.030528,-0.011537,-0.042822,-0.014378,0.031338,0.012297,-0.000038,-0.010264,-0.006781,-0.019950,-0.014435,0.010926,0.024400,0.005318,-0.007983
9,1882,-0.083093,-0.029160,0.013748,0.022716,-0.062732,-0.080670,-0.019723,0.013253,-0.091305,-0.090687,0.030638,-0.113364,-0.021325,0.181508,0.022357


In [ ]:
# Question 5
df_users.loc[df_users['User']==4433, 'Factor10'] 

14   -0.088591
Name: Factor10, dtype: float64

In [ ]:
# Question 6
df_items.sort_values(by=['Factor10'], ascending= False).head(1)

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
65,808,Shrek (2001),-0.241581,-0.449723,0.039779,2.286099,-1.163009,-0.295425,0.905854,0.448446,-2.016609,1.546593,0.513526,0.666029,-0.743928,-0.555847,-0.941293


In [ ]:
def movie_rating(user_id, movie_id):
  user_features = df_users[df_users['User']==user_id].iloc[:,1:]
  item_features = df_items[df_items['Movie ID']==movie_id].iloc[:,2:]
  rating = user_features.dot(item_features.T)
  return rating

In [ ]:
movie_rating(4469, 1637)

,76
8,-0.030868


In [ ]:
movie_rating(2848, 2164)

,82
13,-0.019829


In [ ]:
x = movie_rating(2665, df_items['Movie ID']).transpose()
x['Movie ID'] = df_items['Movie ID']
x['Title'] = df_items['Title']
x.sort_values(by=[23], ascending=False).head(10)

,23,Movie ID,Title
51,0.707718,604,The Matrix Reloaded (2003)
83,0.618237,2501,The Bourne Identity (2002)
84,0.614321,2502,The Bourne Supremacy (2004)
0,0.564326,11,Star Wars: Episode IV - A New Hope (1977)
78,0.534790,1892,Star Wars: Episode VI - Return of the Jedi (1983)
77,0.516400,1891,Star Wars: Episode V - The Empire Strikes Back...
50,0.490787,603,The Matrix (1999)
79,0.477046,1894,Star Wars: Episode II - Attack of the Clones (...
5,0.432130,24,Kill Bill: Vol. 1 (2003)
36,0.394321,393,Kill Bill: Vol. 2 (2004)


In [ ]:
x = movie_rating(3519, df_items['Movie ID']).transpose()
x['Movie ID'] = df_items['Movie ID']
x['Title'] = df_items['Title']
x.sort_values(by=[15], ascending=False).head(10)

,15,Movie ID,Title
77,0.503005,1891,Star Wars: Episode V - The Empire Strikes Back...
0,0.485341,11,Star Wars: Episode IV - A New Hope (1977)
78,0.449368,1892,Star Wars: Episode VI - Return of the Jedi (1983)
9,0.258463,85,Raiders of the Lost Ark (Indiana Jones and the...
59,0.245944,680,Pulp Fiction (1994)
32,0.210365,275,Fargo (1996)
25,0.199212,194,Amelie (2001)
27,0.198369,238,The Godfather (1972)
11,0.178407,105,Back to the Future (1985)
31,0.159434,274,The Silence of the Lambs (1991)
